In [39]:
#0 : Promotional  5.33% 折扣、下殺 特價。。。
#1: Engagement  31.56%   按讚、打卡 分享。。。
#2: Product Awareness  28% (產品) “new product” 系列、首發、新登場、開箱、推薦 新一代 進化版
#3: Brand Awareness 8.44%（品牌）
#Corporate Social Responsibility 5.33%
#Customer Service 3.11% (粉絲) 服務 品管
#Seasonal 18.67%     開學、情人節


In [2933]:
import collections
from collections import OrderedDict

In [772]:
from pymongo import MongoClient
import json
import os.path
from numpy import array

In [35]:
client = MongoClient('mongodb://localhost:27017/')

In [36]:
db = client['fanpage_research']
coll_fanpage = db['fanpage']
coll_post = db['post']
coll_post_detail = db['post_detail']

In [500]:
import numpy as np
import pandas as pd

In [3030]:
import jieba
import jieba.analyse

In [37]:
with open('fid_list_4.json') as d:
    all_fid_list = json.load(d)

In [4235]:
jieba.set_dictionary("dict.txt.big")
stop_words_path = "data/household_stop_words.txt"
jieba.analyse.set_stop_words(stop_words_path)

In [51]:
def pp(o):
    print json.dumps(o,indent=1,ensure_ascii=False)

In [40]:
def get_combined_message_by_label(X_array, y_array, label):
    nd_X = np.array(X_array)
    nd_y = np.array(y_array)
    words = ""
    for p in nd_X[nd_y==label]:
        if p['message'] is not None:
            words = words + p['message']
    return words

In [346]:
def get_keyword(X_array, y_array, label):
    combined_message = get_combined_message_by_label(X_array, y_array, label)
    return jieba.analyse.extract_tags(combined_message, 20)

### -----------------above functions are global functions----------

In [2403]:
class industry_data:
    def __init__(self,industry_name_list,all_fid_list,file_name=None):
        self.name_list = industry_name_list
        self.name_abbr = file_name
        self.all_fid_list = all_fid_list
        self.fid_list = None
        self.posts = None
        self.train_post = None
        self.train_label = None
        self.test_post = None
        self.test_label = None
    def get_name(self):
        name = ""
        if self.name_abbr == None:
            for i in self.name_list:
                name = name + i.rsplit(' ')[0]
            self.name_abbr = name
        return self.name_abbr
    def get_fid_list(self,refresh=False):
        if self.fid_list == None or refresh == True:
            fid_list = []
            for f in self.all_fid_list:
                if f['category'] in self.name_list:
                    fid_list.append(f['fid'])
            self.fid_list = fid_list
        return self.fid_list
    def get_posts(self,refresh=False):
        fid_list = self.get_fid_list()
        if self.posts == None or refresh == True:
            posts = []
            for f in fid_list:
                posts = posts + [p for p in coll_post.find({"fid":f},{'date':False,'_id':False})]
            self.posts = posts
        return self.posts
    def get_train_post_label(self,sample_number=50,refresh=False):
        self.train_label == None
        if self.train_post == None or self.train_label == None or refresh == True:
            file_name = 'data/train_data_'+self.get_name()+'.json'
            if refresh == True or (not os.path.isfile(file_name)):
                tmp_dict = {} 
                posts = self.get_posts()
                tmp_dict['train_X'] = [p for p in np.random.choice(posts, 50, replace=False)]
                tmp_dict['train_y'] = []
                with open(file_name,'w') as outfile:
                    json.dump(tmp_dict,outfile)
            with open(file_name,'r') as infile:
                j_file = json.load(infile)
            try:
                self.train_post = j_file['train_X']
                self.train_label = j_file['train_y']
            except:
                print "error occured when accessing"+file_name+"[train_X] or [train_y]"          
        return self.train_post,self.train_label
    def get_test_post_label(self, sample_number = 50, refresh=False):
        self.test_label == None
        if self.test_post == None or self.test_label == None or refresh == True:
            file_name = 'data/test_data_'+self.get_name()+'.json'
            if refresh == True or (not os.path.isfile(file_name)):
                tmp_dict = {} 
                train_X, train_y = self.get_train_post_label()
                all_posts = self.get_posts()
                posts_without_trainX = [i for i in all_posts if i not in train_X]
                tmp_dict['test_X'] = [p for p in np.random.choice(posts_without_trainX, 50, replace=False)]
                tmp_dict['test_y'] = []
                with open(file_name,'w') as outfile:
                    json.dump(tmp_dict,outfile)
            with open(file_name,'r') as infile:
                j_file = json.load(infile)
            try:
                self.test_post = j_file['test_X']
                self.test_label = j_file['test_y']
            except:
                print "error occured when accessing"+file_name+"[test_X] or [test_y]"          
        return self.test_post,self.test_label    
    def get_keyword_category(self):
        X, y = self.get_train_post_label()
        keyword_dict = collections.OrderedDict()
        if len(X)==len(y) and len(X)!=0:
            for i in range(4):
                combined_message = get_combined_message_by_label(X, y, i)
                keyword_dict['class_'+str(i)] = jieba.analyse.extract_tags(combined_message, 10)
        return keyword_dict

In [4639]:
class Classifier:
    def __init__(self,industry):
        self.name = industry
        self.keyword_dict = None
    def get_keyword_dict(self,refresh=False):
        if self.keyword_dict == None or refresh == True:
            file_name = 'data/keyword_'+self.name+'.json'
            if os.path.isfile(file_name):
                with open(file_name,'r') as infile:
                    keyword_file = json.load(infile, object_pairs_hook=OrderedDict)
                self.keyword_dict = keyword_file
            else:
                print file_name+" doesn't exist"
                return 
        return self.keyword_dict
    def _make_ML_X(self,input_X,print_out=False):
        key_dict = self.get_keyword_dict()
        ML_X = []
        for obj in input_X:
            tmp = []
            match = []
            obj_message = obj['message']
            has_message = 0 if obj_message==None else 1
            has_vivid = 1 if obj['post_type'] in ['album','photo','cover_photo','video','video_inline',
                                                  'image_share','animated_image_share','video_direct_response'] else 0
            for k in key_dict:
                if obj_message!=None:
                    message_jieba = jieba.analyse.extract_tags(obj['message'],0)
                    match = list(set(message_jieba).intersection(key_dict[k]))
                if print_out:
                    print 'match '+k+' : ',','.join(match)
                tmp.append(len(match))
            if print_out:
                print 'has_message : ',has_message
                print 'has_vivid : ',has_vivid
            tmp.append(has_message)
            tmp.append(has_vivid)
            tmp[0] = tmp[0]*5 if tmp[0]!=0 else tmp[0]
            tmp[4] = tmp[4]*5 if tmp[4]!=0 else tmp[4]
            tmp[10] = tmp[10]/3 if tmp[4]!=0 else tmp[10]
            ML_X.append(tmp)
        return ML_X

# Machine Learning Time

In [1006]:
from sklearn import tree

In [1011]:
from sklearn.linear_model import LogisticRegression

In [2404]:
class ML:
    # industry_name is a list obj
    def __init__(self,industry_name_list,all_fid_list,file_name=None):
        self.industry_obj = industry_data(industry_name_list,all_fid_list,file_name)
        self.train_X, self.train_y = self.industry_obj.get_train_post_label()
        self.test_X, self.test_y = self.industry_obj.get_test_post_label()
        self.classifier = Classifier(self.industry_obj.name_abbr)
        self.ML_train_X = self.classifier._make_ML_X(self.train_X)
        self.ML_test_X = self.classifier._make_ML_X(self.test_X)
        self.model_lg = None
        self.model_tree = None
    def get_model_tree(self,refresh=False):
        if self.model_tree == None or refresh == True:
            clf = tree.DecisionTreeClassifier()
            clf_model = clf.fit(np.array(self.ML_train_X),np.array(self.train_y))
            self.model_tree = clf_model
        return self.model_tree
    def get_model_lg(self,refresh=False):
        if self.model_lg == None or refresh == True:
            lg = LogisticRegression()
            lg_model = lg.fit(np.array(self.ML_train_X),np.array(self.train_y))
            self.model_lg = lg_model
        return self.model_lg
    def _predict(self,ML_X,model='lg'):
        prediction = None
        if model == "lg":
            prediction = self.get_model_lg().predict(np.array(ML_X))
        elif model == "tree":
            prediction = self.get_model_tree().predict(np.array(ML_X))
        return prediction
    def accuracy(self,ML_X,y,model = 'lg'):
        prediction = self._predict(ML_X,model)
        accuracy = np.mean(prediction==np.array(y))
        correct_idx = np.where(prediction==np.array(y))[0]
        incorrect_idx = np.where(prediction!=np.array(y))[0]
        print "accuracy:",accuracy
        print "correct_idx:",correct_idx
        print "incorrect_idx:",incorrect_idx 
    def checkOne(self,X,y,index,model='lg'):
        ML_X = self.classifier._make_ML_X([X[index]],print_out = True)
        print "predict class : ", self._predict(np.reshape(np.array(ML_X),(1,-1)),model)
        print "true class : ", y[index]
        print X[index]['message']

# Food

In [2401]:
Food_ML = ML(['Alcohol & Tobacco','Beverages','Restaurants','Food'],all_fid_list)
print Food_ML.accuracy(Food_ML.ML_test_X, Food_ML.test_y,"lg")

accuracy: 0.92
correct_idx: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 48 49]
incorrect_idx: [ 9 22 26 47]
None


In [2037]:
Food_ML.checkOne(Food_ML.test_X,Food_ML.test_y,47,"lg")

match product_test :  
match product_new :  
match info_event :  
match monetary_incentive :  滿額
match product_unit :  
match price_off :  
match engage_other :  
match product_version :  
match info_brand :  
match push_word :  
match engage_join :  
match engage_interaction :  
predict class :  [3]
true class :  0
忙到沒時間、累到想癱著，可是好想吃東西？
外送騎士為您熱食外送，24小時全天候美味不打烊！

4/11前歡樂送滿$398送四塊麥克鷄塊！
麥當勞24hr 歡樂送讓您暖胃也暖心！

滿額雞塊送送送>>http://mcdtw.com/mds/005/ 
立馬訂餐不囉嗦>>http://mcdtw.com/mds/006/


In [1903]:
np.bincount(Food_ML.train_y)

array([11,  5, 10, 24])

# Household + Retail & 不分keyword list (keyword_all.json)

In [2399]:
RetailHouse_ML = ML(['Retail','Household & Personal Care'],all_fid_list)
print RetailHouse_ML.accuracy(RetailHouse_ML.ML_test_X, RetailHouse_ML.test_y,"lg")

accuracy: 0.9
correct_idx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 72 73 74 75 76 77 79 80
 81 82 83 84 85 86 87 88 89 90 91 92 93 97 98]
incorrect_idx: [26 29 52 70 71 78 94 95 96 99]
None


In [1902]:
RetailHouse_ML.checkOne(RetailHouse_ML.test_X,RetailHouse_ML.test_y,2,"lg")

match product_test :  
match product_new :  
match info_event :  
match monetary_incentive :  
match product_unit :  
match price_off :  75折
match engage_other :  
match product_version :  
match info_brand :  
match push_word :  
match engage_join :  
match engage_interaction :  
predict class :  [2]
true class :  2
💢明明很累卻睡不著
今晚又要失眠了嗎？😵
別沮喪！試試這招↓↓
💤睡前用晚安舒眠噴霧
對著枕頭、棉被、臥室
咻！咻！噴個幾下
馬上散發甜橙和薰衣草香氛🍊
整個人變得好放鬆
今晚一定好好睡！

晚安舒眠噴霧2瓶組↘75折
https://goo.gl/QanGwx

轉載自華人健康網

 — Products shown: 晚安舒眠噴霧2瓶組.


In [1830]:
np.bincount(RetailHouse.train_y)

array([16, 19, 36, 29])

# Tech

In [2204]:
tech_md = ML(['Consumer Electronics','Computer Hardware','Computer software','Media & Telecommunications'],all_fid_list)
tech_md.accuracy(tech_md.ML_test_X,tech_md.test_y)

accuracy: 0.9
correct_idx: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18]
incorrect_idx: [ 9 19]


In [2174]:
tech_md.checkOne(tech_md.train_X,tech_md.train_y,19,"lg")

match product_test :  
match product_new :  
match info_event :  
match social_responsibility :  
match monetary_incentive :  超值,優惠
match product_unit :  
match price_off :  
match engage_other :  
match product_version :  
match info_brand :  
match push_word :  
match seasonal :  
match engage_join :  
match engage_interaction :  
match to_fans :  
predict class :  [0]
true class :  0
#母親節優惠預告【專業廚房創意不受限】
傳統爐具總讓你做菜時綁手綁腳？伊萊克斯感應爐超彈性，適用任何形狀的鍋具，盡情發揮烹飪創意😀
快趁即將到來的母親節，為媽媽打造專業廚房💕優惠再贈超值贈品 goo.gl/M2BoCF


In [2170]:
np.bincount(tech_md.train_y)

array([ 7,  8, 45, 20])

In [2155]:
print ','.join(jieba.analyse.extract_tags(tech_md.test_X[7]['message'], 0))

300R,XDP,Pioneer,音樂,播放器,體驗,心得,開箱,Res,Hi,No,耳機,fb,解析,評測,活動,SE,得主,感動,DAC,內外,優選,饗宴,特優,先鋒,聆聽,PIONEER,嘉獎,入耳式,數位,高音質,得獎者,恭喜,佳作,分享,滑鼠,次世代,感受,真章,幻彩,耳罩,海恩,XsyvBd,CX7,升級,gfUUHe,回憶,n3x1PF,新體驗,熱烈,考驗,版電,邁入,溫潤,聽感,無損,鑑定,歷經,愛玩,細膩,得獎,測試報告,gJastK,W4HwBu,贈獎,113690249109653,UkJLT6,MHR5,再創,高竿,APS,電子,殺戮,經典,AP8000,價值,擇一,靈魂,加煒,歡呼,電源,鱔魚,帶來,nsx1693,3ckuma,旗艦級,聽見,redcell6,把關,客邦,多媒體,評選,緋色,7scFO3,astutmomo,耀動,音韻,裡的,Mtcmos,處處,tacototoro,小宅,給得,0w8riv,gUqXW1,擴大,來自,100001304907970,dancer,無遠弗屆,嚴格,質感,別有,CL551,聲音,試用,rMxJhG,行動,5uhiBF,視聽,業界,8bA1EH,隨身,非同凡響,專業,評審,lxqhVu,Jengren,16,此時,Audio,鑑賞,4d1Gcb,帶著,出爐,FNn26J,空間,520evoli,挑色,QCT1K7,hy321250,結果,重低音,忘記,CL722T,PioneerXDP,g8202,兼備,賞的,1485797328116083,頁面,PChome,100000182017563,結過,通話,修羅,有損,純淨,PioneerLife,超乎想像,綜合,0sYmpr,Sjj2uP,細節,Player,標竿,晶片,耳道,出油,影音,音符,工作室,回路,感官,兼具,不凡,更替,加倍,陶醉,美妙,多重,魅力,下列,口袋,均衡,打造,日常,耳朵,注入,令人,平衡,再次,一次


# Automotive, Travel & Leisure, Financial Services,  Courier Services


In [2856]:
auto_ML = ML(["Automotive","Travel & Leisure","Financial Services","Courier Services"],all_fid_list)
auto_ML.accuracy(auto_ML.ML_test_X,auto_ML.test_y)

accuracy: 0.9
correct_idx: [ 0  1  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 27
 28 29]
incorrect_idx: [ 2 13 19]


In [2541]:
auto_ML.checkOne(auto_ML.test_X,auto_ML.test_y,19,"lg")

match product_test :  
match product_new :  
match info_event :  博覽會
match social_responsibility :  
match monetary_incentive :  超值,優惠
match product_unit :  
match price_off :  
match engage_other :  
match product_version :  
match info_brand :  
match push_word :  
match engage_join :  
match engage_interaction :  
match to_fans :  
predict class :  [0]
true class :  3
2017 TTE 台北國際觀光博覽會即將登場！台北君悅酒店旅展獨家 #線上旅展 將於4月1日中午12點鐘搶先開跑，旗下超人氣自助餐凱菲屋、彩日本料理、Ziga Zaga、寶艾西餐廳等多家餐廳超值優惠熱賣，切勿錯過！http://bit.ly/2mVo4se
Pre-order and enjoy discounts up to 35% off at the Grand Hyatt Taipei 2017 Taipei Tourism Expo online sale beginning at noon April 1 !

#餐飲優惠 #便宜餐券 #肋眼牛排 #龍蝦套餐 #義大利餐廳 #台北國際觀光博覽會 #TTE


In [2397]:
np.bincount(auto_ML.train_y)

array([13,  9, 15, 33])

# All dataset

In [4693]:
ALL_ML = ML(["all"],all_fid_list,file_name = "all")

In [4706]:
print ALL_ML.accuracy(ALL_ML.ML_test_X, ALL_ML.test_y,"lg")

accuracy: 0.9
correct_idx: [  0   1   2   3   4   6   8   9  10  11  12  13  14  15  16  17  18  20
  21  23  24  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  61  62  63  64  66  67  68  69  70  72  73  74  77  78  79  80  81
  82  83  84  85  86  87  88  90  91  92  93  94  95  96  97  98  99 100
 101 103 104 105 106 107 108 109 110 111 112 113 114 115 116 119 120 122
 123 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 145 146 147 148 149 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199]
incorrect_idx: [  5   7  19  22  25  60  65  71  75  76  89 102 117 118 121 124 144 150
 151 170]
None


In [4691]:
ALL_ML.ML_train_X[151]  #62

[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 1, 1]

In [4791]:
ALL_ML.checkOne(ALL_ML.test_X,ALL_ML.test_y,0,"lg")

match brand :  
match brand_season :  兒童節
match social_rspns :  
match engage_assist :  
match engage_social :  
match engage_inter :  
match product_new :  
match product_version :  
match product_test :  
match product_unit :  
match promo_deal :  優惠活動,優惠,免費
match promo_discnt :  
has_message :  1
has_vivid :  0
predict class :  [0]
true class :  0
把拔馬麻們注意囉📢📢📢 
🎉 歡慶兒童節🎉 
TGI FRIDAYS Taiwan要請小朋友們吃免費兒童餐喔！ 

4/4兒童節當天用餐並出示此活動頁面👆， 
於指定餐廳 (美麗華、西門、古亭、林森、桃園、 
台中市政、台中英才、台南新天地、高雄夢時代) 
☝消費滿1000元，免費招待1份兒童餐點！ 
✌消費滿2000元，免費招待2份兒童餐點！ (以此類推) 

爸爸媽媽們快點帶家裡的寶貝們到FRIDAYS慶祝兒童節喔！🎉🎈 
各餐廳訂位電話與營業時間請見：http://www.tgifridays.com.tw/store.php

※兒童餐優惠僅提供12歲以下兒童享用。 
※贈送品項需加收原價之10%服務費。 
※不得與EZ Table餐券、網路會員優惠或其它優惠活動同時使用。 
※不得折現或外帶，TGI FRIDAYS保有修改活動辦法之權利


# update class function

In [4692]:
update_class(f_file,"data/train_data_all.json",151,1)

In [4683]:
with open("data/train_data_all.json") as ff:
    f_file = json.load(ff)

In [4684]:
def update_class(loaded_file, file_name, index,new_class):
    loaded_file['train_y'][index] = new_class
    with open(file_name,'w') as o_ff:
        json.dump(loaded_file,o_ff)

# print 特徵向量

In [4794]:
tmpp = []
for i in range(30):
    t = []
    t.append(ALL_ML.test_X[i]['pid'])
    tmpp.append(t+ALL_ML.ML_test_X[i])
    

In [4796]:
pd.DataFrame(data=tmpp,
             columns=['pid','brand','b_season','b_responsibility','eg_assist',"eg_social","eg_interact",
                      "pd_new","pd_ver","pd_try","pd_unit","prm_deal","prm_discount","has_msg","has_vivid"]).iloc[0:1]

,pid,brand,b_season,b_responsibility,eg_assist,eg_social,eg_interact,pd_new,pd_ver,pd_try,pd_unit,prm_deal,prm_discount,has_msg,has_vivid
0,182373771797518_1908464146057096,0,1,0,0,0,0,0,0,0,0,3,0,1,0
